In [39]:
import numpy as np
import random
from scipy.signal import lsim, TransferFunction
import control as ct


def plant_model():
    """
    Returns the transfer function of the plant.
    Example: G(s) = 1 / (s^2 + 2s + 1)
    """
    numerator = [1]
    denominator = [1, 2, 1]
    return TransferFunction(numerator, denominator)


def fitness_function(gains, plant):
    """
    Calculates the fitness of a set of PID gains using ITAE (Integral of Time-multiplied Absolute Error).

    Args:
        gains (list): A list of PID gains [Kp, Ki, Kd].
        plant (TransferFunction): The transfer function of the system.

    Returns:
        float: The fitness value (ITAE).
    """
    Kp, Ki, Kd = gains

    # Check for invalid gains
    if any(g < 0 for g in gains):
        return float('inf')

    # PID controller transfer function C(s)
    # The 'control' library handles the symbolic math more cleanly than scipy for this
    s = ct.TransferFunction([1, 0], [1])
    C_s = Kp + Ki/s + Kd*s

    # Calculate the closed-loop transfer function H(s) = C(s) * G(s) / (1 + C(s) * G(s))
    G_s = ct.TransferFunction(plant.num, plant.den)
    closed_loop = ct.feedback(C_s * G_s)

    # Simulation parameters
    t = np.linspace(0, 50, 501)

    # Simulate a step response using the 'control' library's step_response function
    tout, yout = ct.step_response(closed_loop, T=t)

    # Calculate the error and ITAE
    error = 1 - yout
    itae = np.trapezoid(tout * np.abs(error), tout)

    return itae


def levy_flight(beta=1.5):
    """
    Generates a step size for the Levy flight using the Mantegna algorithm.
    """
    sigma_num = np.random.gamma(1 + beta) * np.sin(np.pi * beta / 2)
    sigma_den = np.random.gamma((1 + beta) / 2) * beta * (2**((beta - 1) / 2))
    sigma = (sigma_num / sigma_den)**(1 / beta)

    u = np.random.normal(0, sigma**2, 3)
    v = np.random.normal(0, 1, 3)

    step = u / np.abs(v)**(1 / beta)
    return step

def cuckoo_search(n_nests, n_iterations, pa, gain_bounds, plant):
    """
    Performs Cuckoo Search to find optimal PID gains.
    """
    # Initialize host nests with random PID gains
    nests = np.random.uniform(gain_bounds[:, 0], gain_bounds[:, 1], size=(n_nests, 3))

    # Evaluate the fitness of each initial nest
    fitness = [fitness_function(nest, plant) for nest in nests]

    # Find the best initial nest
    best_nest_index = np.argmin(fitness)
    best_nest = nests[best_nest_index].copy()
    best_fitness = fitness[best_nest_index]

    # Main Cuckoo Search loop
    for iteration in range(n_iterations):
        # Print the current iteration
        if iteration % 10 == 0:
          print(f"Iteration: {iteration + 1}/{n_iterations}")

        # Generate new cuckoo solutions via Levy flight
        # Using a simple combination for exploration and exploitation
        cuckoo_step = 0.01 * levy_flight() * (best_nest - nests[random.randint(0, n_nests - 1)])
        new_cuckoo = best_nest + cuckoo_step

        # Enforce gain bounds
        for i in range(3):
            new_cuckoo[i] = np.clip(new_cuckoo[i], gain_bounds[i, 0], gain_bounds[i, 1])

        # Evaluate new cuckoo and replace a random nest if better
        new_fitness = fitness_function(new_cuckoo, plant)
        j = random.randint(0, n_nests - 1)
        if new_fitness < fitness[j]:
            nests[j] = new_cuckoo
            fitness[j] = new_fitness

        # Abandon a fraction of the nests (pa)
        for i in range(n_nests):
            if random.random() < pa:
                new_random_nest = np.random.uniform(gain_bounds[:, 0], gain_bounds[:, 1], size=3)
                nests[i] = new_random_nest
                fitness[i] = fitness_function(new_random_nest, plant)

        # Update the best solution found
        current_best_index = np.argmin(fitness)
        if fitness[current_best_index] < best_fitness:
            best_nest = nests[current_best_index].copy()
            best_fitness = fitness[current_best_index]

        # Print best parameters and fitness for this iteration
        if iteration % 10 == 0:
          print(f"  Best Kp: {best_nest[0]:.4f}, Ki: {best_nest[1]:.4f}, Kd: {best_nest[2]:.4f}, Fitness: {best_fitness:.4f}")


    return best_nest


if __name__ == '__main__':

    # Algorithm parameters
    NUM_NESTS = 100
    NUM_ITERATIONS = 100
    PA = 0.25
    GAIN_BOUNDS = np.array([[0.1, 100], [0.1, 100], [0.1, 100]])

    # Get the system to be controlled
    plant = plant_model()

    print("Starting Cuckoo Search for PID tuning...")
    optimal_gains = cuckoo_search(NUM_NESTS, NUM_ITERATIONS, PA, GAIN_BOUNDS, plant)

    Kp_opt, Ki_opt, Kd_opt = optimal_gains
    print("\nOptimal PID Gains:")
    print(f"Kp = {Kp_opt:.4f}")
    print(f"Ki = {Ki_opt:.4f}")
    print(f"Kd = {Kd_opt:.4f}")

Starting Cuckoo Search for PID tuning...
Iteration: 1/100
  Best Kp: 73.4234, Ki: 38.1904, Kd: 23.5575, Fitness: 0.0109
Iteration: 11/100
  Best Kp: 98.0263, Ki: 42.6509, Kd: 44.6200, Fitness: 0.0078
Iteration: 21/100
  Best Kp: 99.9550, Ki: 48.2833, Kd: 52.8107, Fitness: 0.0016
Iteration: 31/100
  Best Kp: 98.5894, Ki: 48.1869, Kd: 52.2152, Fitness: 0.0014
Iteration: 41/100
  Best Kp: 100.0000, Ki: 49.0726, Kd: 52.3590, Fitness: 0.0011
Iteration: 51/100
  Best Kp: 100.0000, Ki: 49.0656, Kd: 52.3130, Fitness: 0.0011
Iteration: 61/100
  Best Kp: 99.8451, Ki: 49.1343, Kd: 51.9996, Fitness: 0.0010
Iteration: 71/100
  Best Kp: 100.0000, Ki: 49.1413, Kd: 52.0058, Fitness: 0.0010
Iteration: 81/100
  Best Kp: 100.0000, Ki: 49.2274, Kd: 51.7163, Fitness: 0.0008
Iteration: 91/100
  Best Kp: 100.0000, Ki: 49.8405, Kd: 51.7131, Fitness: 0.0008

Optimal PID Gains:
Kp = 100.0000
Ki = 49.8405
Kd = 51.7131
